In [7]:
import pandas as pd
import numpy as np
import json

#### Process Data

In [8]:
review_text=[0]*231780
ratings=[0]*231780
itemID=[0]*231780
reviewerID=[0]*231780

i=0
with open("reviews_Video_Games_5.json","r") as json_file:
    for line in json_file:
        data=json.loads(line) 
        review_text[i]=data["reviewText"]
        reviewerID[i]=data["reviewerID"]
        itemID[i]=data["asin"]
        ratings[i]=data["overall"]
        i+=1

reviewerID = pd.DataFrame({'reviewerID': reviewerID})
ratings = pd.DataFrame({'Ratings': ratings})
itemID = pd.DataFrame({'ItemID':itemID})
review_text = pd.DataFrame({'reviewText':review_text})

df_total = pd.concat([ratings,reviewerID,itemID,review_text], axis=1)
   

#### review texts group by item rated 5 stars

In [10]:
items = df_total[['ItemID','reviewText','reviewerID','Ratings']]
items = items[items['Ratings'] == 5]
grouped_item = items.groupby('ItemID')
items = grouped_item['reviewText'].apply("||".join)

In [ ]:
# item_reviews = df_total[['ItemID','reviewText','Ratings']]
# item_reviews = item_reviews[item_reviews['Ratings'] == 5]
# grouped_item = item_reviews.groupby('ItemID')
# item_review=grouped_item['reviewText'].apply(" ".join)

#### review texts group by user rated 5 stars

In [17]:
user_reviews = df_total[['reviewerID','reviewText','Ratings']]
user_reviews = user_reviews[user_reviews['Ratings'] == 5]
grouped_user = user_reviews.groupby('reviewerID')
user_review=grouped_user['reviewText'].apply(" ".join)

#### word2vec on item reviews

In [21]:
from nltk.corpus import stopwords
import re
import nltk.data
from nltk.stem import WordNetLemmatizer
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

def review2wordlist(review):
    #junk words
    junk=['http','www','com']
    #Remove non-letters
    review_text=re.sub("[^a-zA-Z]"," ",review)
    #Convert words to lowercase and form a wordlist
    review_text=review_text.lower().split()
    #remove stop words
    stops=set(stopwords.words("english"))
    review_text=[w for w in review_text if w not in stops]
    review_text=[w for w in review_text if w not in junk]
    #stem words 
    wnl=WordNetLemmatizer()
    review_text=[wnl.lemmatize(w) for w in review_text]
    
    return review_text

#For each review split it into sentence
def review_to_sentences(review,tokenizer):
    sentences=[]
    texts=tokenizer.tokenize(review.strip())
    sentences=[0]*len(texts)
    i=0
    for t in texts:
        if len(t)>0:
            #for each sentence split it into words
            sentence=review2wordlist(t)
            sentences[i]=sentence
            i+=1
    #return list of list
    return sentences

#transform paragraphs to list of lists
for count in range(len(item_review)):
    if count%100==0:
        print(count,len(item_review))
    individual_text=review_to_sentences(item_review[count],tokenizer)
    item_review[count]=individual_text
    

0 10255
100 10255
200 10255
300 10255
400 10255
500 10255
600 10255
700 10255
800 10255
900 10255
1000 10255
1100 10255
1200 10255
1300 10255
1400 10255
1500 10255
1600 10255
1700 10255
1800 10255
1900 10255
2000 10255
2100 10255
2200 10255
2300 10255
2400 10255
2500 10255
2600 10255
2700 10255
2800 10255
2900 10255
3000 10255
3100 10255
3200 10255
3300 10255
3400 10255
3500 10255
3600 10255
3700 10255
3800 10255
3900 10255
4000 10255
4100 10255
4200 10255
4300 10255
4400 10255
4500 10255
4600 10255
4700 10255
4800 10255
4900 10255
5000 10255
5100 10255
5200 10255
5300 10255
5400 10255
5500 10255
5600 10255
5700 10255
5800 10255
5900 10255
6000 10255
6100 10255
6200 10255
6300 10255
6400 10255
6500 10255
6600 10255
6700 10255
6800 10255
6900 10255
7000 10255
7100 10255
7200 10255
7300 10255
7400 10255
7500 10255
7600 10255
7700 10255
7800 10255
7900 10255
8000 10255
8100 10255
8200 10255
8300 10255
8400 10255
8500 10255
8600 10255
8700 10255
8800 10255
8900 10255
9000 10255
9100 10255


In [23]:
# list of list for user reviews
for count in range(len(user_review)):
    if count%1000==0:
        print(count,len(user_review))
    individual_text=review_to_sentences(user_review[count],tokenizer)
    user_review[count]=individual_text

0 22996
1000 22996
2000 22996
3000 22996
4000 22996
5000 22996
6000 22996
7000 22996
8000 22996
9000 22996
10000 22996
11000 22996
12000 22996
13000 22996
14000 22996
15000 22996
16000 22996
17000 22996
18000 22996
19000 22996
20000 22996
21000 22996
22000 22996


#### implement algorithm for word2vec on item reviews

In [24]:
#total reviews to get run by word2vec
from functools import reduce
total_reviews = reduce(lambda x,y: x+y,item_review)

In [26]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
num_features = 300
min_word_count=40
num_workers = 4
context = 10
downsampling = 1e-3

#train the word2vec model
from gensim.models import word2vec

model=word2vec.Word2Vec(total_reviews, workers=num_workers,size=num_features,min_count=min_word_count,window=context,sample=downsampling)
model_name="amazonwordvec"
model.save(model_name)

#### vec2pca

In [27]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

words_array=model.syn0
s = scale(words_array)
pca=PCA(n_components=200)
pca_model=pca.fit_transform(s)
df = pd.DataFrame(pca_model)
df["words"] = model.index2word

//anaconda/envs/virtual/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
//anaconda/envs/virtual/lib/python3.5/site-packages/sklearn/preprocessing/data.py:184: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


#### Make a Dictionary with PCA features

In [28]:
pca_dic={}
features=df.ix[:,0:200]
words=df["words"]
for i in range(len(features)):
    pca_dic[words[i]]=list(np.transpose(features)[i])

#### transform words in each review into vectors of PCA

In [29]:
# Length of each word is now length of PCA

def makeFeatureVec(words, dic, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    pca_wordset = set(dic)
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in pca_wordset: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,dic[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return list(featureVec)


In [30]:
#Transform word list of each paragraph into average feature vectors
item_vector=item_review
for i in range(len(item_vector)):
    if i%1000==0:
        print(i,len(item_vector))
    item_vector[i]=makeFeatureVec(sum(item_vector[i],[]),pca_dic,200) 

0 10255
1000 10255
2000 10255
3000 10255
4000 10255
5000 10255
6000 10255
7000 10255
8000 10255
9000 10255
10000 10255


In [32]:
#Do the same transformation for users
for i in range(len(user_review)):
    if i%1000==0:
        print(i,len(user_review))
    user_review[i]=makeFeatureVec(sum(user_review[i],[]),pca_dic,200) 
    
user_vector=user_review

0 22996
1000 22996
2000 22996
3000 22996
4000 22996
5000 22996
6000 22996
7000 22996
8000 22996
9000 22996
10000 22996
11000 22996
12000 22996
13000 22996
14000 22996
15000 22996
16000 22996
17000 22996
18000 22996
19000 22996
20000 22996
21000 22996
22000 22996


#### Do cosine similarity on the item and user vectors

In [190]:
from scipy import spatial
import heapq

item=[]
i=user_review[3]
for j in item_review:
        result = 1 - spatial.distance.cosine(i, j)
        item.append(result)
tops = heapq.nlargest(15, range(len(item)), item.__getitem__)

In [88]:
x = df_total[['ItemID','reviewText','Ratings']]
x = x[x['Ratings'] == 5]
grouped_x = x.groupby('ItemID')
x=grouped_x['reviewText'].apply(" ".join)

y = df_total[['reviewerID','reviewText','Ratings']]
y = y[y['Ratings'] == 5]
grouped_y = y.groupby('reviewerID')
y=grouped_y['reviewText'].apply(" ".join)

In [17]:
'a' in "abc"

True